# Imports

In [1]:
import psycopg
from psycopg.rows import dict_row

import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.models import (
    Prefetch,
    FusionQuery,
    Filter,
    FieldCondition,
    MatchValue,
)

/Users/eliekawerk/PycharmProjects/00-ai-engineering-bootcamp-cohort-2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Add to Shopping cart Tool

In [5]:
items = [
    {
        "product_id": "B0B96LV4C5",
        "quantity": 2,
    },
    {"product_id": "B0BYYLJRHT", "quantity": 4},
]

In [ ]:
def add_to_shopping_cart(items: list[dict], user_id: str, cart_id: str) -> str:
    """Add a list of provided items to the shopping cart.

    Args:
        items: A list of items to add to the shopping cart. Each item is a dictionary with the following keys: product_id, quantity.
        user_id: The id of the user to add the items to the shopping cart.
        cart_id: The id of the shopping cart to add the items to.

    Returns:
        A list of the items added to the shopping cart.
    """

    conn = psycopg.connect(
        host="localhost",
        port=5433,
        dbname="langgraph_db",
        user="langgraph_user",
        password="langgraph_password",
        autocommit=True,
    )

    with conn.cursor(row_factory=dict_row) as cursor:
        for item in items:
            product_id = item["product_id"]
            quantity = item["quantity"]

            qdrant_client = QdrantClient(url="http://localhost:6333")

            dummy_vector = np.zeros(1536).tolist()
            payload = (
                qdrant_client.query_points(
                    collection_name="Amazon-items-collection-01-hybrid-search",
                    prefetch=[
                        Prefetch(
                            query=dummy_vector,
                            filter=Filter(
                                must=[
                                    FieldCondition(
                                        key="parent_asin",
                                        match=MatchValue(value=product_id),
                                    )
                                ]
                            ),
                            using="text-embedding-3-small",
                            limit=20,
                        )
                    ],
                    query=FusionQuery(fusion="rrf"),
                    limit=1,
                )
                .points[0]
                .payload
            )

            product_image_url = payload.get("image")
            price = payload.get("price")
            currency = "USD"

            # Check if item already exists
            check_query = """
                SELECT id, quantity, price 
                FROM shopping_carts.shopping_cart_items 
                WHERE user_id = %s AND shopping_cart_id = %s AND product_id = %s
            """
            cursor.execute(check_query, (user_id, cart_id, product_id))
            existing_item = cursor.fetchone()

            if existing_item:
                # Update existing item
                new_quantity = existing_item["quantity"] + quantity

                update_query = """
                    UPDATE shopping_carts.shopping_cart_items 
                    SET 
                        quantity = %s,
                        price = %s,
                        currency = %s,
                        product_image_url = COALESCE(%s, product_image_url)
                    WHERE user_id = %s AND shopping_cart_id = %s AND product_id = %s
                    RETURNING id, quantity, price
                """

                cursor.execute(
                    update_query,
                    (
                        new_quantity,
                        price,
                        currency,
                        product_image_url,
                        user_id,
                        cart_id,
                        product_id,
                    ),
                )

            else:
                # Insert new item
                insert_query = """
                    INSERT INTO shopping_carts.shopping_cart_items (
                        user_id, shopping_cart_id, product_id,
                        price, quantity, currency, product_image_url
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s)
                    RETURNING id, quantity, price
                """

                cursor.execute(
                    insert_query,
                    (
                        user_id,
                        cart_id,
                        product_id,
                        price,
                        quantity,
                        currency,
                        product_image_url,
                    ),
                )

    return f"Added {items} to the shopping cart."

In [9]:
user_id = "123"
cart_id = "456"
items = [
    {
        "product_id": "B0B96LV4C5",
        "quantity": 2,
    },
    {"product_id": "B0BYYLJRHT", "quantity": 4},
]

In [10]:
add_to_shopping_cart(items, user_id, cart_id)

"Added [{'product_id': 'B0B96LV4C5', 'quantity': 2}, {'product_id': 'B0BYYLJRHT', 'quantity': 4}] to the shopping cart."

In [11]:
user_id = "123"
cart_id = "456"
items = [{"product_id": "B0BYYLJRHT", "quantity": 4}]

In [22]:
add_to_shopping_cart(items, user_id, cart_id)

"Added [{'product_id': 'B0BYYLJRHT', 'quantity': 4}] to the shopping cart."

# Get Shpping Cart items tool

In [20]:
def get_shopping_cart(user_id: str, cart_id: str) -> list[dict]:
    """
    Retrieve all items in a user's shopping cart.

    Args:
        user_id: User ID
        cart_id: Cart identifier

    Returns:
        List of dictionaries containing cart items
    """
    conn = psycopg.connect(
        host="localhost",
        port=5433,
        dbname="langgraph_db",
        user="langgraph_user",
        password="langgraph_password",
        autocommit=True,
    )
    with conn.cursor(row_factory=dict_row) as cursor:
        query = """
            select 
                product_id, price, quantity,
                currency, product_image_url, 
                (price * quantity) as total_price
            from 
                shopping_carts.shopping_cart_items
            where 
                user_id = %s and shopping_cart_id = %s 
            order by added_at desc
        """
        cursor.execute(query, (user_id, cart_id))
        return [dict(row) for row in cursor.fetchall()]

In [17]:
user_id = "123"
cart_id = "456"

In [21]:
get_shopping_cart(user_id, cart_id)

[{'product_id': 'B0BYYLJRHT',
  'price': Decimal('12.59'),
  'quantity': 8,
  'currency': 'USD',
  'product_image_url': 'https://m.media-amazon.com/images/I/41lk7yDhf3L.jpg',
  'total_price': Decimal('100.72')},
 {'product_id': 'B0B96LV4C5',
  'price': Decimal('14.99'),
  'quantity': 2,
  'currency': 'USD',
  'product_image_url': 'https://m.media-amazon.com/images/I/41J3p7fyJHL.jpg',
  'total_price': Decimal('29.98')}]

In [23]:
get_shopping_cart(user_id, cart_id)

[{'product_id': 'B0BYYLJRHT',
  'price': Decimal('12.59'),
  'quantity': 12,
  'currency': 'USD',
  'product_image_url': 'https://m.media-amazon.com/images/I/41lk7yDhf3L.jpg',
  'total_price': Decimal('151.08')},
 {'product_id': 'B0B96LV4C5',
  'price': Decimal('14.99'),
  'quantity': 2,
  'currency': 'USD',
  'product_image_url': 'https://m.media-amazon.com/images/I/41J3p7fyJHL.jpg',
  'total_price': Decimal('29.98')}]

# Deleting Items from the shopping cart tool

In [26]:
def remove_from_cart(product_id: str, user_id: str, cart_id: str) -> str:
    """
    Remove an item completely from the shopping cart.

    Args:
        user_id: User ID
        product_id: Product ID to remove
        cart_id: Cart identifier

    Returns:
        True if item was removed, False if item wasn't found
    """

    conn = psycopg.connect(
        host="localhost",
        port=5433,
        dbname="langgraph_db",
        user="langgraph_user",
        password="langgraph_password",
        autocommit=True,
    )

    with conn.cursor(row_factory=dict_row) as cursor:
        query = """
                DELETE FROM shopping_carts.shopping_cart_items
                WHERE user_id = %s AND shopping_cart_id = %s AND product_id = %s
            """
        cursor.execute(query, (user_id, cart_id, product_id))

        return cursor.rowcount > 0

In [27]:
remove_from_cart("B0BYYLJRHT", user_id, cart_id)

True